In [ ]:
import netCDF4 as nc
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
import tqdm
import joblib




####需要输入的值有哪些？请输入文件路径：
path_ERA5LWDN = r'Y:\ws\ERA_5\ERA5LWDR'
path_ERA5TCWV = r'Y:\ws\ERA_5\tcwv'
path_ERA5LST = r'Y:\ws\ERA_5\skt'
path=r'Y:\ws'
OUT_PATH=r'Y:\ws\LessRad_LWDR\1998'

os.chdir(path_ERA5LWDN)
file_chdir = os.getcwd()
nc_name1 = [] #csv数据名称（含有.csv扩展名）
for root, dirname, files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.nc':
            nc_name1.append(path_ERA5LWDN + '\\' + file)
            
            
os.chdir(path_ERA5TCWV)
file_chdir = os.getcwd()
nc_name2 = [] #csv数据名称（含有.csv扩展名）
for root, dirname, files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.nc':
            nc_name2.append(path_ERA5TCWV + '\\' + file )

os.chdir(path_ERA5LST)
file_chdir = os.getcwd()
nc_name3 = [] #csv数据名称（含有.csv扩展名）
for root, dirname, files in os.walk(file_chdir):
    for file in files:
        if os.path.splitext(file)[1] == '.nc':
            nc_name3.append(path_ERA5LST + '\\' + file)   
        
          
N = len(nc_name1)
aaaaa = 102
for p in tqdm.trange(0,N):
#####将每天的24个数据一起处理，为了水汽和温度取平均值之后的计算方便。
    filename1 = nc_name1[p]
    with nc.Dataset(filename1) as file1:
        file1.set_auto_mask(False)  # 可选
        variables_1 = {x: file1[x][()] for x in file1.variables}
    A = variables_1['lwdr']/3600
        #####注意 看对应年份经度范围是否需要改变。修正：其实是数据下载的格式问题比如2017年不需要，但2009,2015年就需要
    ERA5_STRD = A
    STORE_LWDN = []
    for k in range(0,24,1):
        T = ERA5_STRD[k,:,:]
        STORE_LWDN.append(T)
    NSTORE_LWDN = np.array(STORE_LWDN)

    
####处理水汽  
    filename2 = nc_name2[p]
    with nc.Dataset(filename2) as file1:
        file1.set_auto_mask(False)  # 可选
        variables_1 = {x: file1[x][()] for x in file1.variables}
    A = variables_1['tcwv']
        #####注意 看对应年份经度范围是否需要改变。修正：其实是数据下载的格式问题比如2017年不需要，但2009,2015年就需要
    ERA5_TCWV = A
    STORE_TCWV = []
    for k in range(0,24,1):
            if k == 0:
                T = ERA5_TCWV[k][:][:]
                STORE_TCWV.append(T)
            else:
                T = (ERA5_TCWV[k-1][:][:]+ERA5_TCWV[k][:][:])/2
                STORE_TCWV.append(T)        
    NSTORE_TCWV = np.array(STORE_TCWV)
#####处理温度   
    filename3 = nc_name3[p]
    with nc.Dataset(filename3) as file1:
        file1.set_auto_mask(False)  # 可选
        variables_1 = {x: file1[x][()] for x in file1.variables}
    A = variables_1['skt']
        #####注意 看对应年份经度范围是否需要改变。修正：其实是数据下载的格式问题比如2017年不需要，但2009,2015年就需要
    ERA5_LST = A  
    STORE_LST = []
    for k in range(0,24,1):
            if k == 0:
                T = ERA5_LST[k][:][:]
                STORE_LST.append(T)
            else:
                T = (ERA5_LST[k-1][:][:] + ERA5_LST[k][:][:])/2
                STORE_LST.append(T)
    NSTORE_LST = np.array(STORE_LST)
    
    
# 处理数据
    X = np.concatenate((NSTORE_LWDN.flatten()[np.newaxis], NSTORE_TCWV.flatten()[np.newaxis], NSTORE_LST.flatten()[np.newaxis]))
    X = np.transpose(X)
    
    
    #加载模型
    loaded_model = joblib.load(os.path.join(path, '2_1.pkl'))
    #使用模型预测
    predicted = loaded_model.predict(X)
    # 将predicted变为三维
    predicted_3d = predicted.reshape((24, 3600, 7200))
    
    

    
  
    date =filename1[-11:-3]
    output_file = os.path.join(OUT_PATH, f'LessRad_LWDR_{date}.nc')
    with nc.Dataset(output_file, 'w', format='NETCDF4') as f:
        # 创建维度和变量
        f.createDimension('time', 24)
        f.createDimension('latitude', 3600)
        f.createDimension('longitude', 7200)
        time_var = f.createVariable('time', 'i4', ('time',))
        lat_var = f.createVariable('latitude', 'f4', ('latitude',))
        lon_var = f.createVariable('longitude', 'f4', ('longitude',))
        lwdr_var = f.createVariable('lwdr', np.int16, 
                                           ('time', 'latitude', 'longitude'), 
                                           fill_value=-999, zlib=True, complevel=9)
        # 设置变量属性
        time_var.units = 'hours since {}-{}-{} 00:00:00'.format(filename1[-11:-7], filename1[-7:-5], filename1[-5:-3])
        time_var.calendar = 'gregorian'
        lat_var.units = 'degrees_north'
        lon_var.units = 'degrees_east'
        lwdr_var.units = 'W/m^2'
        # 写入数据
        time = np.arange(24)
        lon = np.linspace(-180, 180, 7200)
        lat = np.linspace(90, -90, 3600)
        time_var[:] = time
        lat_var[:] = lat
        lon_var[:] = lon
        lwdr_var[:] = predicted_3d
    print(f"数据已保存到文件 {output_file} ")
    
    



             
